In [132]:
import numpy as np
import pandas as pd

def DataPrep(data):
    # Make dummy variables for rank
    data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
    # Delete the previous rank
    data = data.drop('rank', axis=1)
    for field in ['gre', 'gpa']:
        mean, std = data[field].mean(), data[field].std()
        data.loc[:,field] = (data[field]-mean)/std
    # Split off random 10% of the data for testing
    np.random.seed(42)
    sample = np.random.choice(data.index, size=int(len(data)*0.9),replace=False)
    data, test_data = data.ix[sample],data.drop(sample)
    # Split into features and targets
    features_train, targets_train = data.drop('admit',axis=1), data['admit']
    features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']
    
    return [features_train,targets_train,features_test,targets_test]

def sigmoid(x):
    return 1/(1+np.exp(-x))

def BackPropOneLayer(features_train,targets_train,LearnRate,epochs,features_test,targets_test):
    np.random.seed(42)
    n_records,n_features = features_train.shape
    last_loss = None
    last_losstest = None
    # Initialize weights
    weights = np.random.normal(scale=1/n_features**.5, size=n_features)
    for e in range(epochs):
        del_w = np.zeros(weights.shape)
        for x, y in zip(features_train.values,targets_train):
               # Loop through all records, x is the input, y is the target
               # Activation of the output unit
               output = sigmoid(np.dot(x,weights))
               # The error, the target minues the network output
               error = y - output
               # The gradient descent step, the error times the gradient times the inputs, This is the formula to get the
               # Best direction to minimize the error
               del_w += error * output * (1 - output)*x
        weights += LearnRate*del_w/n_records
        # The loop is using a initial weights then updating and appliying again with the new weight and so on
        # Printing out the mean square error on the training set
        if e % (epochs / 10) == 0:
            out = sigmoid(np.dot(features_train, weights))
            loss = np.mean((out - targets_train) ** 2)
            if last_loss and last_loss < loss:
                print("Train loss: ", loss, "  WARNING - Loss Increasing")
            else:
                print("Train loss: ", loss)
            last_loss = loss
            
    tes_out = sigmoid(np.dot(features_test, weights))
    # One or Zero
    predictions = tes_out > 0.5
    accuracy = np.mean(predictions == targets_test)
    print("Prediction accuracy: {:.3f}".format(accuracy))
    return 0

def BakPropMoreLayer(features_train,targets_train,LearnRate,epochs,features_test,targets_test,n_hidden):
     n_records,n_features = features_train.shape
     last_loss = None
     # Initialize weights
     weights_input_hidden = np.random.normal(scale=1/n_features ** -.5,size=(n_features, n_hidden))
     weights_hidden_output = np.random.normal(scale=1 / n_features ** -.5,size=n_hidden)
     for e in range(epochs):
         # The vectors are the hidden (6,Unit hidden layer)  and the output a vector (unite hidden layer)
         del_w_input_hidden = np.zeros(weights_input_hidden.shape)
         del_w_hidden_output = np.zeros(weights_hidden_output.shape)
         for x, y in zip(features_train.values, targets_train):
             hidden_input = np.dot(x, weights_input_hidden)
             hidden_activations = sigmoid(hidden_input)
             output = sigmoid(np.dot(hidden_activations,weights_hidden_output))
             ## Backward pass ##
             # TODO: Calculate the error
             error = y - output
             # TODO: Calculate error gradient in output unit the sigmoid gradiet
             output_error = error * output * (1 - output)
             
             hidden_error = np.dot(output_error, weights_hidden_output)*hidden_activations * (1 - hidden_activations)
             
             del_w_hidden_output += output_error * hidden_activations
             # The x[:, None] transform to a matrix (6,1)
             del_w_input_hidden += hidden_error * x[:, None]
         weights_input_hidden += LearnRate * del_w_input_hidden / n_records
         weights_hidden_output += LearnRate * del_w_hidden_output / n_records
         # Printing out the mean square error on the training set
         if e % (epochs / 10) == 0:
                hidden_activations = sigmoid(np.dot(features_train, weights_input_hidden))
                out = sigmoid(np.dot(hidden_activations,weights_hidden_output))
                loss = np.mean((out - targets_train) ** 2)
                if last_loss and last_loss < loss:
                    print("Train loss: ", loss, "  WARNING - Loss Increasing")
                else:
                    print("Train loss: ", loss)
                last_loss = loss
                
     hidden = sigmoid(np.dot(features_test, weights_input_hidden))
     out = sigmoid(np.dot(hidden, weights_hidden_output))
     predictions = out > 0.5
     accuracy = np.mean(predictions == targets_test)
     print("Prediction accuracy: {:.3f}".format(accuracy))
     return 0 



In [162]:
np.random.seed(50)
admissions = pd.read_csv('TestScore.csv')
dataset = DataPrep(admissions)
#lr 0.6
learnrate = 0.6
x = dataset[0]
y = dataset[1]
xtest = dataset[2]
ytest = dataset[3]
ep = 3000
#OneLay = BackPropOneLayer(x,y,learnrate,ep,xtest,ytest)
MoreThanOneLayer = BakPropMoreLayer(x,y,learnrate,ep,xtest,ytest,10)

Train loss:  0.365736380539
Train loss:  0.208997218062
Train loss:  0.196185234326
Train loss:  0.192816645321
Train loss:  0.191028696139
Train loss:  0.189782793074
Train loss:  0.188774989314
Train loss:  0.1879137881
Train loss:  0.187164076343
Train loss:  0.186506570542
Prediction accuracy: 0.775
